In [1]:
#conda install -c conda-forge pydotplus00
#conda install -c gafortiby sklearn-pandas 

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import numpy as np

random_seed = 123
np.random.seed(random_seed)

# 1. Reading the data



Histone modifications play an important role in affecting gene regulation. Specific histone modifications at specific locations in or near the gene can alter the expression of genes. Predicting gene expression from histone modification signals is a widely studied research topic.

In this competition you will predict gene expression levels (low=0, high=1) based on the presence of histone modifications at specific locations in the gene. You will try to find the model that learns the true underlying model best.

For each gene a region of 10.000bp around the transcription start site of the gene is extracted (5000bp upstream and 5000bp downstream). This region is binned in 100 bins of 100bp. For each bin five core histone modification marks are counted [1].

The dataset is compiled from the "E047" (Primary T CD8+ naive cells from peripheral blood) celltype from Roadmap Epigenomics Mapping Consortium (REMC) database.

[1] Kundaje, A. et al. Integrative analysis of 111 reference human epige-
nomes. Nature, 518, 317–330, 2015.

We start by loading the Pandas library and reading the datasets into Pandas DataFrames:

In [3]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_test.csv")
test_label = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data_test_label.csv")

Let's look at the first 5 rows of the trainset:

In [4]:
train.head(5)

,GeneId,H3K4me3_0,H3K4me1_0,H3K36me3_0,H3K9me3_0,H3K27me3_0,H3K4me3_1,H3K4me1_1,H3K36me3_1,H3K9me3_1,H3K27me3_1,H3K4me3_2,H3K4me1_2,H3K36me3_2,H3K9me3_2,H3K27me3_2,H3K4me3_3,H3K4me1_3,H3K36me3_3,H3K9me3_3,H3K27me3_3,H3K4me3_4,H3K4me1_4,H3K36me3_4,H3K9me3_4,H3K27me3_4,H3K4me3_5,H3K4me1_5,H3K36me3_5,H3K9me3_5,H3K27me3_5,H3K4me3_6,H3K4me1_6,H3K36me3_6,H3K9me3_6,H3K27me3_6,H3K4me3_7,H3K4me1_7,H3K36me3_7,H3K9me3_7,...,H3K4me1_92,H3K36me3_92,H3K9me3_92,H3K27me3_92,H3K4me3_93,H3K4me1_93,H3K36me3_93,H3K9me3_93,H3K27me3_93,H3K4me3_94,H3K4me1_94,H3K36me3_94,H3K9me3_94,H3K27me3_94,H3K4me3_95,H3K4me1_95,H3K36me3_95,H3K9me3_95,H3K27me3_95,H3K4me3_96,H3K4me1_96,H3K36me3_96,H3K9me3_96,H3K27me3_96,H3K4me3_97,H3K4me1_97,H3K36me3_97,H3K9me3_97,H3K27me3_97,H3K4me3_98,H3K4me1_98,H3K36me3_98,H3K9me3_98,H3K27me3_98,H3K4me3_99,H3K4me1_99,H3K36me3_99,H3K9me3_99,H3K27me3_99,Label
0,7008,1,1,2,1,3,1,2,5,0,1,1,3,8,5,3,0,2,8,2,6,1,0,4,3,2,0,0,2,1,0,2,1,7,5,0,0,1,6,1,...,14,30,7,4,1,16,21,6,1,1,12,31,4,4,0,10,31,9,1,1,17,23,3,2,2,8,16,6,3,2,11,15,5,2,2,10,12,2,2,1
1,9839,0,0,1,1,0,2,0,2,0,2,2,0,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,3,0,0,0,5,2,1,0,0,1,1,2,2,1,2,2,1,4,1,2,1,0,2,0,0,1,1,0,0,1,1,0,0,0,1,1,0,0,0,1
2,11972,7,3,1,1,1,4,2,1,1,2,2,2,4,1,0,1,1,0,0,0,4,1,0,1,1,8,2,1,2,1,4,2,2,3,0,1,2,0,2,...,0,3,0,0,2,0,2,1,2,3,2,3,3,2,3,1,0,1,2,2,0,0,1,3,2,0,1,1,1,1,1,0,2,0,1,1,1,1,0,0
3,14705,4,2,4,1,3,0,2,1,1,3,1,6,1,1,1,1,1,0,1,1,1,1,1,0,1,2,3,1,1,0,1,1,1,2,2,1,1,2,1,...,5,0,2,0,0,3,3,1,1,2,2,0,0,1,5,2,2,1,2,8,1,0,1,1,2,3,1,2,2,3,1,2,4,2,4,3,0,0,0,0
4,12058,1,1,2,0,8,0,2,1,1,3,0,0,0,2,3,1,3,0,1,10,1,3,0,1,8,0,0,0,3,5,0,3,2,0,2,1,0,0,0,...,1,6,0,1,2,0,3,1,0,2,1,1,1,0,1,4,4,2,2,0,5,1,1,2,2,2,0,0,0,2,1,2,2,1,0,1,1,1,0,1


There is a column called `GeneId` that identifies the gene. This column should not be used as a feature. 

The label for each datapoint is in the `Label` column.

In [5]:
train_ids = train.pop("GeneId")
train_labels = train.pop("Label")

Now `train` contains the feature columns only.

Let's look at the number datapoints in each class:

In [6]:
train_labels.value_counts()

1    5250
0    5186
Name: Label, dtype: int64

Let's look at `test`:

In [7]:
test.head(5)

,GeneId,H3K4me3_0,H3K4me1_0,H3K36me3_0,H3K9me3_0,H3K27me3_0,H3K4me3_1,H3K4me1_1,H3K36me3_1,H3K9me3_1,H3K27me3_1,H3K4me3_2,H3K4me1_2,H3K36me3_2,H3K9me3_2,H3K27me3_2,H3K4me3_3,H3K4me1_3,H3K36me3_3,H3K9me3_3,H3K27me3_3,H3K4me3_4,H3K4me1_4,H3K36me3_4,H3K9me3_4,H3K27me3_4,H3K4me3_5,H3K4me1_5,H3K36me3_5,H3K9me3_5,H3K27me3_5,H3K4me3_6,H3K4me1_6,H3K36me3_6,H3K9me3_6,H3K27me3_6,H3K4me3_7,H3K4me1_7,H3K36me3_7,H3K9me3_7,...,H3K4me3_92,H3K4me1_92,H3K36me3_92,H3K9me3_92,H3K27me3_92,H3K4me3_93,H3K4me1_93,H3K36me3_93,H3K9me3_93,H3K27me3_93,H3K4me3_94,H3K4me1_94,H3K36me3_94,H3K9me3_94,H3K27me3_94,H3K4me3_95,H3K4me1_95,H3K36me3_95,H3K9me3_95,H3K27me3_95,H3K4me3_96,H3K4me1_96,H3K36me3_96,H3K9me3_96,H3K27me3_96,H3K4me3_97,H3K4me1_97,H3K36me3_97,H3K9me3_97,H3K27me3_97,H3K4me3_98,H3K4me1_98,H3K36me3_98,H3K9me3_98,H3K27me3_98,H3K4me3_99,H3K4me1_99,H3K36me3_99,H3K9me3_99,H3K27me3_99
0,5222,2,2,7,2,0,3,2,9,2,0,1,7,8,1,0,0,2,5,4,0,0,2,5,2,0,2,4,10,3,0,0,4,10,1,0,1,2,6,2,...,1,1,2,0,3,3,1,1,0,1,2,2,0,2,1,3,1,0,2,1,2,0,1,0,1,0,0,0,1,0,0,2,1,1,0,0,3,1,1,1
1,891,1,2,0,0,1,0,4,1,2,4,1,2,2,3,4,1,1,2,3,1,2,2,3,1,1,1,0,0,0,0,0,0,0,0,1,2,0,0,0,...,0,3,0,2,3,0,1,1,3,0,1,0,1,1,3,1,0,0,0,1,1,0,0,0,0,3,3,0,1,0,3,2,2,2,0,4,2,5,2,1
2,7219,1,1,2,4,4,2,2,1,1,1,2,0,0,3,1,3,0,0,2,1,2,1,2,1,0,1,1,2,0,1,0,1,1,2,0,2,1,1,3,...,2,0,1,0,1,2,0,1,1,1,2,2,1,2,2,2,1,2,2,0,0,2,0,1,2,3,4,1,1,0,1,3,1,1,0,0,1,1,1,0
3,7225,1,5,2,4,1,0,8,3,4,0,0,3,2,2,0,0,0,1,0,0,2,0,1,2,1,2,2,2,2,2,1,5,0,0,0,1,4,2,2,...,2,7,0,4,2,0,2,0,4,1,0,2,1,2,1,0,3,0,3,0,1,0,0,2,2,0,1,0,0,4,0,2,1,1,2,1,1,0,2,0
4,9432,1,16,3,2,2,2,7,0,4,2,0,5,1,1,2,0,4,1,0,2,0,4,1,1,2,0,3,2,2,2,0,1,2,1,3,1,5,2,0,...,1,0,1,1,0,3,1,2,1,1,2,0,3,3,0,4,2,3,2,1,4,2,2,1,3,3,0,0,0,3,0,1,2,0,1,1,1,1,2,1


This is a blind test so the `eyeDetection` column is not available in the testset. The testset does contain a column called `index` that you will need to send your predictions to the Kaggle website.

Let's look at the shape of the train- and testset:

In [8]:
print(train.shape)
print(test.shape)

(10436, 500)
(5049, 501)


We can get some general statistics about the trainset using the DataFrame `.describe()` function:

In [9]:
train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
H3K4me3_0,10436.0,1.482081,1.883112,0.0,0.0,1.0,2.0,21.0
H3K4me1_0,10436.0,1.738885,2.098565,0.0,0.0,1.0,2.0,24.0
H3K36me3_0,10436.0,2.342564,3.798700,0.0,0.0,1.0,3.0,45.0
H3K9me3_0,10436.0,1.887792,5.292531,0.0,0.0,1.0,2.0,104.0
H3K27me3_0,10436.0,1.134438,1.455765,0.0,0.0,1.0,2.0,41.0
...,...,...,...,...,...,...,...,...
H3K4me3_99,10436.0,1.455922,1.842602,0.0,0.0,1.0,2.0,27.0
H3K4me1_99,10436.0,1.738022,2.134836,0.0,0.0,1.0,2.0,35.0
H3K36me3_99,10436.0,2.397183,3.892699,0.0,0.0,1.0,3.0,35.0
H3K9me3_99,10436.0,1.826945,4.924786,0.0,0.0,1.0,2.0,115.0


# 2. Data pre-processing




We can use the Pandas `boxplot()` function to plot the feature values:

In [11]:
plt.figure(figsize=(22,8))
train.boxplot()
plt.show()

KeyboardInterrupt: ignored

In [ ]:
marks = {}
for m in train.columns:
  marks[m.split("_")[0]] = True
marks = list(marks.keys())
marks

In [ ]:
for mark in marks:
  cols = []
  for m in train.columns:
    if mark in m:
      cols.append(m)
  plt.figure(figsize=(22,8))    
  train[cols].boxplot()
  plt.show()

In this case I choose scaling the features to [0,1]:

In [12]:
from sklearn import preprocessing

scaler_minmax = preprocessing.MinMaxScaler()
scaler_minmax.fit(train)
train_norm = pd.DataFrame(scaler_minmax.transform(train),columns=train.columns)
train_norm.head()

,H3K4me3_0,H3K4me1_0,H3K36me3_0,H3K9me3_0,H3K27me3_0,H3K4me3_1,H3K4me1_1,H3K36me3_1,H3K9me3_1,H3K27me3_1,H3K4me3_2,H3K4me1_2,H3K36me3_2,H3K9me3_2,H3K27me3_2,H3K4me3_3,H3K4me1_3,H3K36me3_3,H3K9me3_3,H3K27me3_3,H3K4me3_4,H3K4me1_4,H3K36me3_4,H3K9me3_4,H3K27me3_4,H3K4me3_5,H3K4me1_5,H3K36me3_5,H3K9me3_5,H3K27me3_5,H3K4me3_6,H3K4me1_6,H3K36me3_6,H3K9me3_6,H3K27me3_6,H3K4me3_7,H3K4me1_7,H3K36me3_7,H3K9me3_7,H3K27me3_7,...,H3K4me3_92,H3K4me1_92,H3K36me3_92,H3K9me3_92,H3K27me3_92,H3K4me3_93,H3K4me1_93,H3K36me3_93,H3K9me3_93,H3K27me3_93,H3K4me3_94,H3K4me1_94,H3K36me3_94,H3K9me3_94,H3K27me3_94,H3K4me3_95,H3K4me1_95,H3K36me3_95,H3K9me3_95,H3K27me3_95,H3K4me3_96,H3K4me1_96,H3K36me3_96,H3K9me3_96,H3K27me3_96,H3K4me3_97,H3K4me1_97,H3K36me3_97,H3K9me3_97,H3K27me3_97,H3K4me3_98,H3K4me1_98,H3K36me3_98,H3K9me3_98,H3K27me3_98,H3K4me3_99,H3K4me1_99,H3K36me3_99,H3K9me3_99,H3K27me3_99
0,0.047619,0.041667,0.044444,0.009615,0.073171,0.023810,0.095238,0.119048,0.00,0.04,0.047619,0.12,0.195122,0.041667,0.125000,0.00,0.074074,0.186047,0.017094,0.272727,0.047619,0.000000,0.074074,0.024194,0.068966,0.000000,0.000000,0.047619,0.009091,0.000000,0.095238,0.047619,0.155556,0.05,0.000000,0.00,0.055556,0.15,0.009091,0.066667,...,0.045455,0.636364,0.714286,0.071429,0.133333,0.045455,0.64,0.552632,0.060606,0.038462,0.041667,0.666667,0.574074,0.039216,0.133333,0.000000,0.454545,0.704545,0.081818,0.033333,0.052632,0.566667,0.547619,0.027523,0.071429,0.086957,0.333333,0.326531,0.058252,0.103448,0.10,0.500000,0.394737,0.050505,0.068966,0.074074,0.285714,0.342857,0.017391,0.046512
1,0.000000,0.000000,0.022222,0.009615,0.000000,0.047619,0.000000,0.047619,0.00,0.08,0.095238,0.00,0.024390,0.000000,0.041667,0.05,0.000000,0.023256,0.000000,0.000000,0.047619,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.00,0.000000,0.05,0.000000,0.00,0.000000,0.000000,...,0.136364,0.136364,0.000000,0.000000,0.000000,0.227273,0.08,0.026316,0.000000,0.000000,0.041667,0.055556,0.037037,0.019608,0.033333,0.090909,0.090909,0.022727,0.036364,0.033333,0.105263,0.033333,0.000000,0.018349,0.000000,0.000000,0.041667,0.020408,0.000000,0.000000,0.05,0.045455,0.000000,0.000000,0.000000,0.037037,0.028571,0.000000,0.000000,0.000000
2,0.333333,0.125000,0.022222,0.009615,0.024390,0.095238,0.095238,0.023810,0.01,0.08,0.095238,0.08,0.097561,0.008333,0.000000,0.05,0.037037,0.000000,0.000000,0.000000,0.190476,0.041667,0.000000,0.008065,0.034483,0.421053,0.095238,0.023810,0.018182,0.037037,0.190476,0.095238,0.044444,0.03,0.000000,0.05,0.111111,0.00,0.018182,0.000000,...,0.045455,0.000000,0.071429,0.000000,0.000000,0.090909,0.00,0.052632,0.010101,0.076923,0.125000,0.111111,0.055556,0.029412,0.066667,0.136364,0.045455,0.000000,0.009091,0.066667,0.105263,0.000000,0.000000,0.009174,0.107143,0.086957,0.000000,0.020408,0.009709,0.034483,0.05,0.045455,0.000000,0.020202,0.000000,0.037037,0.028571,0.028571,0.008696,0.000000
3,0.190476,0.083333,0.088889,0.009615,0.073171,0.000000,0.095238,0.023810,0.01,0.12,0.047619,0.24,0.024390,0.008333,0.041667,0.05,0.037037,0.000000,0.008547,0.045455,0.047619,0.041667,0.018519,0.000000,0.034483,0.105263,0.142857,0.023810,0.009091,0.000000,0.047619,0.047619,0.022222,0.02,0.066667,0.05,0.055556,0.05,0.009091,0.100000,...,0.045455,0.227273,0.000000,0.020408,0.000000,0.000000,0.12,0.078947,0.010101,0.038462,0.083333,0.111111,0.000000,0.000000,0.033333,0.227273,0.090909,0.045455,0.009091,0.066667,0.421053,0.033333,0.000000,0.009174,0.035714,0.086957,0.125000,0.020408,0.019417,0.068966,0.15,0.045455,0.052632,0.040404,0.068966,0.148148,0.085714,0.000000,0.000000,0.000000
4,0.047619,0.041667,0.044444,0.000000,0.195122,0.000000,0.095238,0.023810,0.01,0.12,0.000000,0.00,0.000000,0.016667,0.125000,0.05,0.111111,0.000000,0.008547,0.454545,0.047619,0.125000,0.000000,0.008065,0.275862,0.000000,0.000000,0.000000,0.027273,0.185185,0.000000,0.142857,0.044444,0.00,0.066667,0.05,0.000000,0.00,0.000000,0.100000,...,0.045455,0.045455,0.14285

In [ ]:
for mark in marks:
  cols = []
  for m in train_norm.columns:
    if mark in m:
      cols.append(m)
  plt.figure(figsize=(22,8))    
  train_norm[cols].boxplot()
  plt.show()

# 3. Building a decision tree

The scikit-learn `DecisionTreeClassifier` class computes a decision tree predictive model from a dataset. 

To get all the options for learning you can simply type: 

In [13]:
from sklearn.tree import DecisionTreeClassifier
help(DecisionTreeClassifier)

Help on class DecisionTreeClassifier in module sklearn.tree._classes:

class DecisionTreeClassifier(sklearn.base.ClassifierMixin, BaseDecisionTree)
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : {"gini", "entropy"}, default="gini"
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  splitter : {"best", "random"}, default="best"
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to choose the best split and "random" to choose
 |      the best random split.
 |  
 |  max_depth : int, default=None
 |      The maximum depth of the tree. If None, then nodes are expanded until
 |      all leaves are pure or until all leaves contain less than
 |      min_samples_split samples.
 |  
 |  min_samples_split : int or float, default=2
 |      The minimum 

You notice that there are many (hyper)parameters to set. These influence the complexity of the model. An important such parameter is the `max_depth` that set a limit on how deep a decision tree can become. 

Let's create a decision tree model with `max_depth=3`:

In [14]:
cls_DT = DecisionTreeClassifier(max_depth=3)

This creates a decision tree model with default values for the other hyperparameters:

In [ ]:
cls_DT

Now we can fit the trainset in just one line of code:

In [16]:
cls_DT.fit(train_norm,train_labels)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Let's see how well the decision tree performs on the trainset:

In [17]:
from sklearn.metrics import accuracy_score

predictions = cls_DT.predict(train_norm)
print("Accuracy: %f"%(accuracy_score(predictions, train_labels)))

Accuracy: 0.825125


These are the predictions for the first 100 data points:

In [ ]:
predictions[0:100]

For the Kaggle competition your predictions are not evaluated by accuracy, but by log-loss:

$$ - \frac{1}{N} \sum_{i=1}^N [y_{i} \log \, p_{i} + (1 - y_{i}) \log \, (1 - p_{i})],$$

where $N$ is the number of datapoints, $y_i$ is the label of datapoint $i$, and $p_i$ is the prediction of the model expressed as a probability.

Let's compute the log-loss:


In [18]:
from sklearn.metrics import log_loss

print("Log-loss: %f"%log_loss(train_labels,predictions))

Log-loss: 6.040057


This is a very high log-loss (note that we want the log-loss the be as close to zero as possible).

The reason is that we don't actually provide probabilities but just the classes 0 and 1. Any error will be punished with a very high contribution to the log-loss. Therefor, we should predict class probabilities rather than just classes. 

Decision trees allow us to do that:

In [19]:
from sklearn.metrics import log_loss

predictions = cls_DT.predict_proba(train_norm)

print("Log-loss: %f"%log_loss(train_labels,predictions))

Log-loss: 0.452246


Now the log-loss is much smaller.

The following code plots the fitted decision tree `cls` as a `tree.png` file:

In [ ]:
"""
from sklearn import tree
from io import StringIO
from IPython.display import Image, display
import pydotplus

out = StringIO()
tree.export_graphviz(cls_DT, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png("tree.png")
"""

# 4. Kaggle evaluation

Let's make a Kaggle submission:

In [20]:
#code for submission
index_col = test.pop("GeneId")
predictions_test = cls_DT.predict_proba(test)

out_tmp = pd.DataFrame()
out_tmp["GeneId"] = index_col
out_tmp["eyeDetection"] = predictions_test[:,1]
out_tmp.to_csv("submission_DT.csv",index=False)

How does the model perform on the testset?

# 5. Hyperparamters

For our first submission we set the hyperparameter `max_depth=3`. Other values might result in lower log-loss on the testset. 

Since we don't have the testset labels we can only check this on the public leaderboard, which we should not do!

So, we need to create our own testset (**not seen during training!**) with known class labels.

Scikit-learn offers many options to do this. One of them is the `train_test_split` function:

In [29]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.2, random_state=random_seed)

#train fold
print(train_X.shape)
print(train_y.shape)
#validation fold
print(val_X.shape)
print(val_y.shape)

(8348, 500)
(8348,)
(2088, 500)
(2088,)


Fit a decision tree model with default paramters on the `train_X` data set.

In [30]:
#solution
cls_DT = DecisionTreeClassifier(max_depth=14)
cls_DT.fit(train_X,train_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=14, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

What is the accuracy on `train_X`? 

In [34]:
#solution
predictions = cls_DT.predict(train_X)
print("Accuracy: %f"%(accuracy_score(predictions, train_y)))
predictions = cls_DT.predict_proba(train_X)
print("Log-loss: %f"%(log_loss(train_y,predictions[:,1])))

Accuracy: 0.954001
Log-loss: 0.155349


What is the accuracy on `val_X`?

In [35]:
#solution
predictions = cls_DT.predict(val_X)
print("Accuracy: %f"%(accuracy_score(predictions, val_y)))
predictions = cls_DT.predict_proba(val_X)
print("Log-loss: %f"%(log_loss(val_y,predictions[:,1])))

Accuracy: 0.805556
Log-loss: 4.407348


What do you see?


The following code evaluates different values for this hyperparameter.

In [40]:
for maxdepth in range(1,20,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    cls.fit(train_X,train_y)
    predictions_train = cls.predict(train_X)
    predictions_val = cls.predict(val_X)
    predictions_train_prob = cls.predict_proba(train_X)[:,1]
    predictions_val_prob = cls.predict_proba(val_X)[:,1]
    print("%i (%f) %f (%f) %f"%(maxdepth,
                                accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y),
                                log_loss(train_y,predictions_train_prob),log_loss(val_y,predictions_val_prob)))

1 (0.787254) 0.795019 (0.517571) 0.507348
2 (0.810134) 0.811782 (0.484475) 0.479824
3 (0.822353) 0.818008 (0.456446) 0.461147
4 (0.825587) 0.818487 (0.427133) 0.456196
5 (0.840920) 0.820402 (0.401390) 0.590077
6 (0.851102) 0.826628 (0.378458) 0.761725
7 (0.864878) 0.827586 (0.355997) 0.850584
8 (0.878773) 0.828544 (0.327821) 1.028743
9 (0.894705) 0.817529 (0.291779) 1.647373
10 (0.910278) 0.812739 (0.260448) 2.016281
11 (0.925491) 0.817529 (0.225833) 2.566585
12 (0.939027) 0.811782 (0.193870) 3.449689
13 (0.947532) 0.810345 (0.171298) 3.849344
14 (0.954121) 0.802203 (0.152804) 4.598492
15 (0.961548) 0.806992 (0.135780) 4.777776
16 (0.963464) 0.808908 (0.128609) 4.734535
17 (0.966459) 0.805556 (0.117869) 5.178614
18 (0.970292) 0.805077 (0.106601) 5.326464
19 (0.971730) 0.801724 (0.100003) 5.535981


In [45]:
for run in range(5):
  train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.8, random_state=run)
  min_m = 100
  best = None
  for maxdepth in range(1,20,1):
      cls = DecisionTreeClassifier(max_depth=maxdepth)
      cls.fit(train_X,train_y)
      predictions_val_prob = cls.predict_proba(val_X)[:,1]
      m = log_loss(val_y,predictions_val_prob)
      if m < min_m:
        min_m = m
        best = maxdepth
  print("%i %f"%(best,min_m))

2 0.489871
2 0.501669
3 0.479379
2 0.491435
3 0.479280


In [50]:
from sklearn.model_selection import cross_val_predict

for maxdepth in range(1,20,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    predictions = cross_val_predict(cls,train_norm,train_labels,
                                    cv=10,
                                    method="predict_proba")
    print("%i %f"%(maxdepth,log_loss(train_labels,predictions[:,1])))

1 0.518693
2 0.488489
3 0.469868
4 0.458350
5 0.509028
6 0.623721
7 0.856069
8 1.143359
9 1.498481
10 2.002092
11 2.603200
12 3.051423
13 3.532863
14 4.041015
15 4.432267
16 4.737619
17 4.963082
18 5.359835
19 5.440707


In [54]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':range(1,20)
    }

GSCV = GridSearchCV(cls_DT, params,
                    cv=10,
                    scoring="neg_log_loss",
                    verbose=1).fit(train_norm,train_labels)

print(GSCV.best_estimator_)
print(GSCV.best_score_)

Fitting 10 folds for each of 19 candidates, totalling 190 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: ignored

Play with the hyperparameters in a Template notebook and make some Kaggle submissions.

# 5. Ensemble learning: bagging

We have seen that bias and variance play an important role in Machine Learning. Let's first see what bagging can do for our data set. 

In [ ]:
from sklearn.ensemble import BaggingClassifier

cls_bagged = BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=random_seed)
                                                            
cls_bagged.fit(train_X,train_y)
predictions_train = cls_bagged.predict(train_X)
predictions_val = cls_bagged.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

With the `RandomForestClassifier` the variance of the decision tree is reduced also by selecting features for decision tree contruction at random. Let's see how far we get with default hyperparameter values.   

In [28]:
from sklearn.ensemble import RandomForestClassifier

cls_RF = RandomForestClassifier(random_state=random_seed)
cls_RF.fit(train_X,train_y)
predictions_train = cls_RF.predict(train_X)
predictions_val = cls_RF.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

0.999521 0.864464


Now let's see what we can do with the `RandomForestClassifier`.

In [ ]:
help(RandomForestClassifier)

What happens when you increase the `n_estimators` hyperparameter? 

In [ ]:
#solution

# 6. Ensemble learning: boosting

How about the `GradientBoostingClassifier`?

In [ ]:
#solution
from sklearn.ensemble import GradientBoostingClassifier

cls_GB = GradientBoostingClassifier(random_state=random_seed,max_depth=10)
cls_GB.fit(train_norm,train_filtered_label)
predictions_train = cls_GB.predict(train_norm)
predictions_val = cls_GB.predict(test_norm)
print("%f %f"%(accuracy_score(predictions_train, train_filtered_label),accuracy_score(predictions_val, test_label)))

# 7. Logistic regression

Before we evaluate other Machine Learning algorithms such as logistic regression we need to look at the distributions and scales of the features. In fact, this should always be the first thing to do when analyzing a new data set.

In Pandas we can create a boxplot for each of the features as follows:

In [ ]:
X.boxplot()

What do we see?

There are many approaches to outlier detection and removal. Let's use a simple one:

In [ ]:
from scipy import stats

print(X.shape)
tmp = (np.abs(stats.zscore(X)) < 3).all(axis=1)
X_clean = X[tmp]
y_clean = y[tmp]
print(X_clean.shape)

In [ ]:
X_clean.boxplot()

This should look much better.

However, feature values between features are still at very different scales. We need to normalize them. 

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(X_clean)
X_norm = pd.DataFrame(scaler.transform(X_clean),columns=X_clean.columns)

In [ ]:
X_norm.boxplot()

Now each feature is at a similar scale.

Let's start with `LogisticRegression`.

In [ ]:
from sklearn.linear_model import LogisticRegression

train_X, val_X, train_y, val_y = train_test_split(X_norm,y_clean,test_size=.2, random_state=random_seed)

cls_LR=LogisticRegression()
cls_LR.fit(train_X,train_y)
predictions_train = cls_LR.predict(train_X)
predictions_val = cls_LR.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

Does this perform well?

What could be the reason?

# 8. Non-linear transformations

What does linear classification mean?

In [ ]:
import seaborn as sns

dataset_tmp = pd.read_csv('svm_example1.csv')

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})

In [ ]:
import imp
compomics_import = imp.load_source('compomics_import', 'compomics_import.py')

X_tmp = dataset_tmp.copy()
y_tmp = X_tmp.pop('y')

cls_LR.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(cls_LR)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

polynomial_features = PolynomialFeatures(degree=2)
model = Pipeline([("polynomial_features", polynomial_features),
                         ("LR", cls_LR)])
model.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(model)

What happens when you increase the degree?

In [ ]:
#solution

Would this work on our dataset?

In [ ]:
#solution 

Kaggle submission?

The following code ranks the features based on the value of the hyperparameters:

In [ ]:
indices = np.argsort(cls_LR.coef_[0])[::-1]
fnames = poly.get_feature_names()
for i in range(len(fnames)):
    print("%f\t%s"%(cls_LR.coef_[0][indices[i]],fnames[indices[i]]))


What is wrong with this code?

In [ ]:
#solution

# 9. Support Vector Machines

Let's see what an SVM can do with an RBF kernel:

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='poly',C=1,degree=2)
model.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(model)

In [ ]:
dataset2_tmp = pd.read_csv("svm_example2.csv")
X_tmp = dataset2_tmp.copy()
y_tmp = X_tmp.pop('y')

model = SVC(kernel='rbf',C=1,gamma=1)
model.fit(X_tmp,y_tmp)

sns.lmplot(x="x1", y="x2", data=dataset2_tmp, hue='y', markers=['o','+'], 
           fit_reg=False, height=5, scatter_kws={"s": 80})
compomics_import.plot_svm_decision_function(model)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X_norm,y_clean,test_size=.2, random_state=random_seed)

cls_SVM = SVC(kernel='rbf')
cls_SVM.fit(train_X,train_y)
predictions_train = cls_SVM.predict(train_X)
predictions_val = cls_SVM.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

# 10. Neural Networks

How about a Neural Network?

In [ ]:
from sklearn.neural_network import MLPClassifier

cls_MLP=MLPClassifier(hidden_layer_sizes=(60,10))
cls_MLP.fit(train_X,train_y)
predictions_train = cls_MLP.predict(train_X)
predictions_val = cls_MLP.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

Is feature normalization required?

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X,y,test_size=.2, random_state=random_seed)
cls_MLP.fit(train_X,train_y)
predictions_train = cls_MLP.predict(train_X)
predictions_val = cls_MLP.predict(val_X)
print("%f %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))